In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

print(pd.read_csv('data/bgcs/all_features_PFAM_ID.csv').shape)
pd.read_csv('data/bgcs/all_features_PFAM_ID.csv')['BGC_id'].nunique()

(235, 953)


235

In [4]:

le = LabelEncoder()

# df_merged = pd.read_csv('data/acc/all_features_CDS_motifs.csv')
# df_merged = pd.read_csv('data/acc/all_features_CDS_smCOG.csv')
# df_merged = pd.read_csv('data/acc/all_features_PFAM_desc.csv')
# df_merged = pd.read_csv('data/acc/all_features_PFAM_GO.csv')
# df_merged = pd.read_csv('data/acc/all_features_PFAM_ID.csv')

df_dict = {}

folder = 'data/acc/'

df_dict['CDS_motifs'] = pd.read_csv(folder + 'all_features_CDS_motifs.csv')
df_dict['CDS_motifs'] = df_dict['CDS_motifs'].rename(columns = {df_dict['CDS_motifs'].columns[0]: 'index'}).drop(df_dict['CDS_motifs'].iloc[:, 1:6], axis=1)
df_dict['CDS_smCOG'] = pd.read_csv(folder + 'all_features_CDS_smCOG.csv')
df_dict['CDS_smCOG'] = df_dict['CDS_smCOG'].rename(columns = {df_dict['CDS_smCOG'].columns[0]: 'index'}).drop(df_dict['CDS_smCOG'].columns[[1]], axis=1)
df_dict['PFAM_GO'] = pd.read_csv(folder + 'all_features_PFAM_GO.csv')
df_dict['PFAM_GO'] = df_dict['PFAM_GO'].rename(columns = {df_dict['PFAM_GO'].columns[0]: 'index'}).drop(df_dict['PFAM_GO'].columns[[1]], axis=1)
df_dict['PFAM_ID'] = pd.read_csv(folder + 'all_features_PFAM_ID.csv')
df_dict['PFAM_ID'] = df_dict['PFAM_ID'].rename(columns = {df_dict['PFAM_ID'].columns[0]: 'index'}).drop(df_dict['PFAM_ID'].columns[[1]], axis=1)

df_merged = None

for df in df_dict:
    if df_merged is None:
        df_merged = df_dict[df]
    else:
        df_merged = pd.merge(df_merged, df_dict[df], on = 'index', how = 'inner')

df_merged = df_merged.drop("index", axis=1)
df_merged.loc[:,'is_antifung'] = 0

df_bgcs = pd.read_csv('data/bgcs/dataset_merged.csv')

cols = list(set(df_bgcs.columns) & set(df_merged.columns))

df_bgcs = df_bgcs[cols]

df_merged = df_merged[cols]

#df_merged = pd.concat([df_merged, df_bgcs]).reset_index(drop=True)

df_merged

,C1_dual_004-017,SMCOG1091:glutamine-binding lipoprotein glnH,GO:0008299,GO:0006260,GO:0003824,SMCOG1047:phosphopantetheine-binding domain-containing protein,GO:0006098,SMCOG1287:AraC family transcriptional regulator,GO:0016627,GO:0016887,...,SMCOG1048:sensor histidine kinase,PKSI-KS_m3,SMCOG1288:ABC transporter related protein,GO:0008661,"SMCOG1284:transcriptional regulator, MarR family",GO:0016620,C2_LCL_024-062,C5_DCL_263-294,GO:0019290,GO:0016773
0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,3.0,0.0,...,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,1.0,...,0.0,14.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,...,0.0,1.0,1.0,0.0,0.0,2.0,14.0,7.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
239,0.0,1.0,2.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
240,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
241,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Antifungical

In [5]:
steps = [('scaler', StandardScaler()), ('model', SVC(C = 10, gamma = 0.001, kernel = 'rbf', probability = True, random_state = 42))]
model = Pipeline(steps = steps)

y_train = df_bgcs.pop('is_antifung')
X_train = df_bgcs
y_test = df_merged.pop('is_antifung')
X_test = df_merged

scores_acc = cross_val_score(model, X_train, y_train, cv = 10, scoring = 'accuracy', n_jobs = -1)
scores_auc = cross_val_score(model, X_train, y_train, cv = 10, scoring = 'roc_auc', n_jobs = -1)


print(y_train.value_counts())
print(y_test.value_counts())
print('ACC: {:.4f}, AUC: {:.4f}'.format(scores_acc.mean(), scores_auc.mean()))

0    200
1     18
Name: is_antifung, dtype: int64
0    243
Name: is_antifung, dtype: int64
ACC: 0.9219, AUC: 0.6800


In [143]:
model.fit(X_train, y_train)

predict = model.predict(X_test)

print(accuracy_score(y_test, predict))

1.0


Antifungi

In [26]:
y = df_merged['is_antifung']

scores_acc = cross_val_score(model, X, y, cv = 10, scoring = 'accuracy', n_jobs = -1)
scores_auc = cross_val_score(model, X, y, cv = 10, scoring = 'roc_auc', n_jobs = -1)

print('ACC: {:.4f}, AUC: {:.4f}'.format(scores_acc.mean(), scores_auc.mean()))

ACC: 0.9268, AUC: 0.7025


In [27]:
y.value_counts()

0    200
1     18
Name: is_antifung, dtype: int64

Cytoxic

In [4]:
y = df_merged['cytotoxic']

scores_acc = cross_val_score(model, X, y, cv = 10, scoring = 'accuracy', n_jobs = -1)
scores_auc = cross_val_score(model, X, y, cv = 10, scoring = 'roc_auc', n_jobs = -1)

print('ACC: {:.4f}, AUC: {:.4f}'.format(scores_acc.mean(), scores_auc.mean()))

ACC: 0.8853, AUC: 0.7774
